Charger les 2 fichiers CSV inventaires dans des dataframes pandas.

In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

In [2]:
DATA_DIR = Path("../../data")
INPUT_DIR = Path(DATA_DIR, "portfolio-inventories")
OUTPUT_DIR = Path(DATA_DIR, "output")

In [3]:
PORTFOLIO_CODE = "Portoflio Code"
ASSET_CODE = "Asset Code"
QUANTITY = "Quantity"

In [4]:
COLS = [PORTFOLIO_CODE, ASSET_CODE, QUANTITY]

In [5]:
portfolio_d1_df = pd.read_csv(
    Path(INPUT_DIR, "day1.ptf_inventories.csv"),
    sep=";",
    # usecols=COLS,
)

portfolio_d2_df = pd.read_csv(
    Path(INPUT_DIR, "day2.ptf_inventories.csv"),
    sep=";",
    # usecols=COLS,
)


In [6]:
portfolio_d1_df = portfolio_d1_df[COLS]
portfolio_d2_df = portfolio_d2_df[COLS]

In [7]:
pd.set_option("display.max_columns", None)
display(portfolio_d1_df.head())
display(portfolio_d2_df.head())

,Portoflio Code,Asset Code,Quantity
0,111111,frw01G8,7.247088e+04
1,111111,frw01UH,1.272192e+06
2,111111,frw01VY,5.294401e+04
3,111111,frw01WK,2.101907e+06
4,111111,frw01XE,4.907177e+05


,Portoflio Code,Asset Code,Quantity
0,111111,frw01G8,7.247088e+04
1,111111,frw01UH,1.272192e+06
2,111111,frw01VY,5.294401e+04
3,111111,frw01WK,2.101907e+06
4,111111,frw01ZR,5.972627e+04


Explorer ces fichiers pour essayer d'en comprendre la structure.

Déterminer s'il y a des lignes en "doublon".

In [8]:
duplicated_data_mask = portfolio_d1_df.duplicated(
    [PORTFOLIO_CODE, ASSET_CODE],
    keep=False
)

In [9]:
portfolio_d1_df[duplicated_data_mask]

,Portoflio Code,Asset Code,Quantity
47,111111,FutUCO22412,31.609809
62,111111,FutBCO22412,-2.370736
69,111111,FutBCO22412,11.063433
70,111111,FutUCO22412,42.673242
274,111111,ISINS7620S9Y,303.454162
275,111111,ISINS7620S9Y,1296.792395
371,111111,ISIN01Q91500,2245.876896
372,111111,ISIN01Q91500,6513.991291
597,222222,FutBCO22412,-1.580490
602,222222,FutPIN22412,2.370736


Agréger les lignes multiples associées à un même couple (portfolio, asset), en utilisant une fonction d'agrégation pertinente.

In [10]:
aggregated_d1_df = portfolio_d1_df.groupby(
    [PORTFOLIO_CODE, ASSET_CODE]
).sum(
).reset_index()

aggregated_d2_df = portfolio_d2_df.groupby(
    [PORTFOLIO_CODE, ASSET_CODE]
).sum(
).reset_index()

In [11]:
aggregated_d1_df.head()

,Portoflio Code,Asset Code,Quantity
0,111111,CashAUD,95178.18449
1,111111,CashCAD,14870.54203
2,111111,CashCHF,34195.61730
3,111111,CashDKK,87008.76382
4,111111,CashEUR,181067.57390


In [12]:
aggregated_d2_df.head()

,Portoflio Code,Asset Code,Quantity
0,111111,CashAUD,95178.18449
1,111111,CashCAD,14870.54203
2,111111,CashCHF,34195.61730
3,111111,CashDKK,87008.76382
4,111111,CashEUR,181067.57390


Ecrire un petit script permettant de calculer les variations de chaque asset de chaque portfolio, dès lors qu'il y a eu une variation non nulle. Stocker les résultats dans un dataframe.

In [13]:
merged_df = aggregated_d1_df.merge(
    aggregated_d2_df,
    how="outer",
    on=[PORTFOLIO_CODE, ASSET_CODE],
    suffixes=("_before", "_after"),
).fillna(0)

In [14]:
merged_df["variation"] = merged_df["Quantity_after"] - merged_df["Quantity_before"]

In [15]:
merged_df

,Portoflio Code,Asset Code,Quantity_before,Quantity_after,variation
0,111111,CashAUD,9.517818e+04,9.517818e+04,0.0
1,111111,CashCAD,1.487054e+04,1.487054e+04,0.0
2,111111,CashCHF,3.419562e+04,3.419562e+04,0.0
3,111111,CashDKK,8.700876e+04,8.700876e+04,0.0
4,111111,CashEUR,1.810676e+05,1.810676e+05,0.0
...,...,...,...,...,...
1166,222222,frw06GY,2.256224e+04,2.256224e+04,0.0
1167,222222,frw079D,9.939006e+06,9.939006e+06,0.0
1168,222222,frw07BR,9.645461e+05,9.645461e+05,0.0
1169,222222,frw07KD,7.421405e+05,7.421405e+05,0.0


In [16]:
variation_df = merged_df[merged_df["variation"] != 0][[PORTFOLIO_CODE, ASSET_CODE, "variation"]]

In [17]:
variation_df.head()

,Portoflio Code,Asset Code,variation
45,111111,ISIN00018578,6875.133354
55,111111,ISIN00069035,7033.182396
61,111111,ISIN00096472,22601.013090
66,111111,ISIN01759635,-79024.521310
84,111111,ISIN07750454,-102731.877700


Exporter le dataframe des variations au format CSV.

In [19]:
variation_df.to_csv(Path(OUTPUT_DIR, "variation.csv"), index=False)

### Compléments

Enregistrer le résultat dans un fichier Excel templaté.

In [22]:
from datetime import datetime
from shutil import copyfile

from openpyxl import load_workbook

In [23]:
template_path = Path(DATA_DIR, "templates", "variation_template.xlsx")

today = datetime.today().strftime("%Y-%m-%d")
output_file_path = Path(OUTPUT_DIR, f"filled_template_variations_{today}.xlsx")

output_file_path.parent.mkdir(parents=True, exist_ok=True)
copyfile(template_path, output_file_path)

wb = load_workbook(output_file_path)

with pd.ExcelWriter(
    output_file_path,
    engine="openpyxl",
    mode="a",
    if_sheet_exists="overlay",
) as writer:

    variation_df.to_excel(
        writer,
        sheet_name="Data",
        startrow=1,
        index=False,
        header=False,
    )  

Mettre en forme un nouveau fichier Excel dans le code.

In [24]:
from openpyxl.styles import Font

In [25]:
today = datetime.today().strftime("%Y-%m-%d")
output_file = Path(OUTPUT_DIR, f"red_header_variations_{today}.xlsx")

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    variation_df.to_excel(
        writer,
        sheet_name="Data",
        startrow=0,
        startcol=0,
        index=False,
)

    wb = writer.book
    ws = writer.sheets["Data"]

    red_bold = Font(bold=True, color="FF0000")

    for cell in ws[1]:
        cell.font = red_bold

### Visualisations

Visualiser la distribution des variations non nulles.

In [ ]:
# Votre code ici

Visualiser la composition du portefeuille 111111.

In [ ]:
# Votre code ici

Regrouper les assets représentant moins de 2% dans une catégorie "others".

In [ ]:
# Votre code ici

S'il n'y en a pas, rajouter des bordures entre les quarties du camembert. Exporter le graphique dans un fichier image.

In [ ]:
# Votre code ici

Exporter le graphique dans un fichier image.

In [42]:
# Votre code ici